In [ ]:
#Importamos librerias necesarias 
from pyspark.sql import SparkSession, functions as F

In [ ]:
# Inicializa la sesión de Spark 
spark = SparkSession.builder.appName('Tarea3').getOrCreate()
spark.conf.set("spark.sql.debug.maxToStringFields", 100)

In [ ]:
# Define la ruta del archivo .csv en HDFS 
file_path = 'hdfs://localhost:9000/Tarea3/rows.csv'
# Lee el archivo .csv 
df = spark.read.format('csv').option('header','true').option('inferSchema', 'true').load(file_path)

In [ ]:
#imprimimos el esquema 
df.printSchema()
# Muestra las primeras filas del DataFrame 
df.show()

In [ ]:
# Estadisticas básicas 
df.summary().show()

In [ ]:
# Consulta: Filtrar por valor asignado mayor a 50000000 y seleccionar columnas relevantes
print("Subsidios con valor mayor a 50000000\n")
subsidios_filtrados = df.filter(F.col('Valor Asignado') > 50000000).select(
    'Valor Asignado', 'Año de Asignación', 'Departamento', 'Municipio')
subsidios_filtrados.show()

In [ ]:
# Ordenar filas por los valores en la columna "Valor Asignado" en orden descendente
print("Subsidios ordenados de mayor a menor por valor asignado\n")
subsidios_ordenados = df.sort(F.col("Valor Asignado").desc())
subsidios_ordenados.show()

In [ ]:
# Contar cuántos subsidios se asignaron por año
print("Número de subsidios asignados por año\n")
subsidios_por_anio = df.groupBy('Año de Asignación').count().orderBy('Año de Asignación')
subsidios_por_anio.show()

In [ ]:
# Suma total del valor asignado por departamento
print("Suma total de valor asignado por departamento\n")
valor_por_departamento = df.groupBy('Departamento').agg(F.sum('Valor Asignado').alias('Total Asignado')).orderBy(F.col('Total Asignado').desc())
valor_por_departamento.show()

In [ ]:
# Filtrar subsidios asignados a un departamento específico
departamento_filtro = 'Antioquia'  # Puedes cambiar el valor
estado_filtro = 'Asignados' # Puesdes cambiar el valor
print(f"Subsidios asignados en {departamento_filtro}\n")
subsidios_departamento = df.filter((F.col('Departamento') == departamento_filtro)&(F.col('Estado de Postulación') == estado_filtro))
subsidios_departamento.show()

In [ ]:
# Identificar el municipio con mayor cantidad de subsidios
print("Municipio con mayor cantidad de subsidios asignados\n")
municipio_mas_subsidios = df.groupBy('Municipio').count().orderBy(F.col('count').desc()).limit(1)
municipio_mas_subsidios.show()

In [ ]:
# Valor promedio de los subsidios por año
print("Valor promedio de subsidios por año\n")
valor_promedio_anio = df.groupBy('Año de Asignación').agg(F.avg('Valor Asignado').alias('Valor Promedio')).orderBy('Año de Asignación')
valor_promedio_anio.show()

In [ ]:
# Filtrar subsidios por un rango de años (Ejemplo: 2015-2020)
print("Subsidios asignados entre 2015 y 2020\n")
subsidios_rango_anio = df.filter((F.col('Año de Asignación') >= 2015) & (F.col('Año de Asignación') <= 2020))
subsidios_rango_anio.show()